<a href="https://colab.research.google.com/github/mohammadbadi/CrimeAnalytics_Clustering/blob/main/Code%20Sections/5.9%20Best%20Model%20Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **5.9 Best Model Training**

In [3]:
import warnings                                                                   # Import necessary libraries
import numpy as np
import pandas as pd
import ast                                                                        # For safely evaluating strings
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler
from IPython.display import display, HTML
from google.colab import files

print("\n\n")
warnings.filterwarnings("ignore", category=DeprecationWarning)                    # Ignore Deprecation Warnings
warnings.filterwarnings("ignore", category=FutureWarning)                         # Ignore future warnings

url = "https://raw.githubusercontent.com/mohammadbadi/CrimeAnalytics_Clustering/refs/heads/main/Output_CSV/FE_Encoded.csv"  # Read the dataset from CSV file
url1 = "https://raw.githubusercontent.com/mohammadbadi/CrimeAnalytics_Clustering/refs/heads/main/Output_CSV/Feature_Combo_Current_Results.csv"  # Load feature combinations
original_data = pd.read_csv(url)
feature_combos = pd.read_csv(url1)

                                                                                  # Debugging: Check input files for missing values
html_output = """
<p style="color: black; font-size: 16px; font-weight: bold;">
    Checking input files...<br>
    Missing values in original_data: <span style="color: green;">{orig_missing}</span><br>
    Missing values in feature_combos: <span style="color: green;">{feat_missing}</span>
</p>
"""
display(HTML(html_output.format(orig_missing=original_data.isnull().sum().sum(), feat_missing=feature_combos.isnull().sum().sum())))

original_data['_id'] = original_data.index                                        # Store _id before clustering
sample_data = original_data.copy()
#sample_data = original_data.sample(frac=0.1, random_state=42)                     # Use 10% of the data (for now)

# Define feature sets
set_names = ['4_Set_165','4_Set_369', '4_Set_490', '4_Set_494', '4_Set_495']      # Define the set names to match
feature_sets = []                                                                 # Initialize an empty list to store feature sets
for set_name in set_names:                                                        # Iterate through the set names
    matched_features = feature_combos[feature_combos['Feature Set'] == set_name]['Feature_Names_String']  # Extract corresponding feature sets
    if not matched_features.empty:
        features_list = ast.literal_eval(matched_features.values[0])              # Convert string to list
        feature_sets.append(features_list)
    else:
        feature_sets.append([])                                                   # Handle missing feature sets

                                                                                  # Debugging: Check feature set validity
debug_feature = "<p style=\"color: black; font-size: 16px; font-weight: bold;\">Checking feature set validity.<br>"
for i, features in enumerate(feature_sets, start=1):
    valid_features = [f for f in features if f in sample_data.columns]
    debug_feature += f"<span style=\"color: darkblue;\">{set_names[i-1]}</span>: <span style=\"color: green;\">{len(valid_features)}</span> valid features out of <span style=\"color: green;\">{len(features)}</span><br>"
    if len(valid_features) == 0:
        debug_feature += f"Warning: Feature set <span style=\"color: darkblue;\">{set_names[i-1]}</span> has no valid features!<br>"
debug_feature += "</p>"
display(HTML(debug_feature))

                                                                                  # Debugging: Standardization Check for each feature set
debug_standard = "<p style=\"color: black; font-size: 16px; font-weight: bold;\">Standardization Checks:<br>"
for i, features in enumerate(feature_sets, start=1):
    valid_features = [f for f in features if f in sample_data.columns]
    if valid_features:
        data_for_clustering = sample_data[valid_features].copy()
        numerical_cols = data_for_clustering.select_dtypes(include=['int64', 'float64']).columns.tolist()
        if len(numerical_cols) == 0:
            debug_standard += f" Warning: Feature set <span style=\"color: darkblue;\">{set_names[i-1]}</span> has NO numerical columns for clustering!<br>"
        else:
            scaler = StandardScaler()
            try:
                scaled_data = scaler.fit_transform(data_for_clustering[numerical_cols])
                debug_standard += f" Scaling successful for <span style=\"color: darkblue;\">{set_names[i-1]}</span> (<span style=\"color: green;\">{len(numerical_cols)}</span> numerical features).<br>"
            except Exception as e:
                debug_standard += f" Error scaling <span style=\"color: darkblue;\">{set_names[i-1]}</span>: {e}<br>"
debug_standard += "</p>"
display(HTML(debug_standard))

                                                                                  # Debugging: Check _id mapping
sample_ids = sample_data['_id'].values                                            # Store _id for mapping
duplicated_ids = sample_data['_id'].duplicated().sum()
html_id = """
<p style="color: black; font-size: 16px; font-weight: bold;">
    Checking _id mapping...<br>
    Total unique IDs: <span style="color: green;">{unique}</span>, Duplicates: <span style="color: green;">{dups}</span>
</p>
"""
display(HTML(html_id.format(unique=len(set(sample_ids)), dups=duplicated_ids)))
if duplicated_ids > 0:
    display(HTML("<p style=\"color: red; font-size: 16px; font-weight: bold;\">Warning: Duplicate _id values found!</p>"))

display(HTML("<p style=\"color: darkblue; font-size: 16px; font-weight: bold;\">Pre-clustering checks completed. Data is ready for clustering!</p>"))

                                                                                  # Prepare clustering results DataFrame
clustering_results = original_data.copy()                                         # Create a copy to store clustering results
for i in range(1, 6):                                                             # Add placeholder columns for clustering results for each set
    for algo in ['KMeans','DBSCAN']:
        clustering_results[f'{algo}{i}_Cluster'] = -1
        clustering_results[f'{algo}{i}_Silhouette_Score'] = np.nan
        clustering_results[f'{algo}{i}_Davies_Bouldin_Index'] = np.nan
        if algo == "KMeans":
            clustering_results[f'{algo}{i}_Calinski_Harabasz_Score'] = np.nan
        clustering_results[f'{algo}{i}_Prediction_Accuracy'] = np.nan

                                                                                  # Clustering for each feature set with debugging outputs
debug_cluster = "<p style=\"color: darkblue; font-size: 18px; font-weight: bold;\">Clustering Debug Info:<br>"
for i, features in enumerate(feature_sets, start=1):                              # Perform clustering on each feature set
    valid_features = [f for f in features if f in sample_data.columns]
    data_for_clustering = sample_data[valid_features].copy()
    sample_ids = sample_data['_id'].values                                        # Store _id for mapping back
    numerical_cols = data_for_clustering.select_dtypes(include=['int64', 'float64']).columns.tolist()
    scaler = StandardScaler()
    data_scaled = pd.DataFrame(scaler.fit_transform(data_for_clustering[numerical_cols]), columns=numerical_cols)

    kmeans = KMeans(n_clusters=4, random_state=42)                                # KMeans Clustering
    kmeans_labels = kmeans.fit_predict(data_scaled)
    silhouette_score_kmeans = silhouette_score(data_scaled, kmeans_labels)
    davies_bouldin_score_kmeans = davies_bouldin_score(data_scaled, kmeans_labels)
    calinski_harabasz_score_kmeans = calinski_harabasz_score(data_scaled, kmeans_labels)
    kmeans_accuracy = max(0, silhouette_score_kmeans) * 100

    dbscan = DBSCAN(eps=0.5, min_samples=5)                                       # DBSCAN Clustering
    dbscan_labels = dbscan.fit_predict(data_scaled)
    silhouette_score_dbscan = -1 if len(set(dbscan_labels)) <= 1 else silhouette_score(data_scaled, dbscan_labels)
    davies_bouldin_score_dbscan = -1 if len(set(dbscan_labels)) <= 1 else davies_bouldin_score(data_scaled, dbscan_labels)
    dbscan_accuracy = max(0, silhouette_score_dbscan) * 100

    debug_cluster += f"Feature set <span style=\"color: darkblue;\">{set_names[i-1]}</span> - KMeans labels: <span style=\"color: green;\">{kmeans_labels[:10]}</span> ...<br>"
    debug_cluster += f"Feature set <span style=\"color: darkblue;\">{set_names[i-1]}</span> - DBSCAN labels: <span style=\"color: green;\">{dbscan_labels[:10]}</span> ...<br>"

    for idx, original_idx in enumerate(sample_ids):                               # Map clustering results back to original data based on feature _id
        clustering_results.loc[original_idx, f'KMeans{i}_Cluster'] = kmeans_labels[idx]
        clustering_results.loc[original_idx, f'KMeans{i}_Silhouette_Score'] = silhouette_score_kmeans
        clustering_results.loc[original_idx, f'KMeans{i}_Davies_Bouldin_Index'] = davies_bouldin_score_kmeans
        clustering_results.loc[original_idx, f'KMeans{i}_Calinski_Harabasz_Score'] = calinski_harabasz_score_kmeans
        clustering_results.loc[original_idx, f'KMeans{i}_Prediction_Accuracy'] = kmeans_accuracy
        clustering_results.loc[original_idx, f'DBSCAN{i}_Cluster'] = dbscan_labels[idx]
        clustering_results.loc[original_idx, f'DBSCAN{i}_Silhouette_Score'] = silhouette_score_dbscan
        clustering_results.loc[original_idx, f'DBSCAN{i}_Davies_Bouldin_Index'] = davies_bouldin_score_dbscan
        clustering_results.loc[original_idx, f'DBSCAN{i}_Prediction_Accuracy'] = dbscan_accuracy

debug_cluster += "</p>"
display(HTML(debug_cluster))

                                                                                  # Debugging: Check for NaN values in clustering_results and print a few rows
debug_nan = "<ul style=\"color: darkblue; font-size: 18px; font-weight: bold;\"><li>Checking for NaN values in clustering_results: <span style=\"color: green;\">{nan_dict}</span></li><li>Preview of clustering_results (first 10 rows): <span style=\"color: green;\">{preview}</span></li></ul>"
nan_dict = clustering_results.isnull().sum().to_dict()
preview = clustering_results.head(10).to_html(classes="table table-bordered", index=False)
display(HTML(debug_nan.format(nan_dict=nan_dict, preview=preview)))

clustering_results.to_csv('Best_Clustering_Models.csv', index=False)                  # Save the clustering results to a CSV file
display(HTML("""
    <p style="color: darkblue; font-size: 18px; font-weight: bold;">
         Clustering results saved as <span style="color: darkblue;">'clustering_results.csv'</span>.
    </p>
"""))

                                                                                  # Define the base columns (first file)
base_columns = ['_id', 'EVENT_UNIQUE_ID', 'OCC_YEAR', 'OCC_MONTH', 'OCC_DAY', 'OCC_DOY', 'OCC_DOW', 'OCC_HOUR', 'DIVISION', 'LOCATION_TYPE', 'PREMISES_TYPE', 'HOOD_158', 'NEIGHBOURHOOD_158', 'LONG_WGS84', 'LAT_WGS84', 'OCC_DATETIME', 'reporting_delay_days', 'reporting_delay_hours', 'Location_Engineered', 'Hood_158_Encoded', 'Division_Encoded', 'Location_Engineered_Other', 'Location_Engineered_Public', 'Location_Engineered_Residential', 'OCC_Month_Encoded', 'OCC_DOW_Encoded']
clustering_results_base = clustering_results[base_columns]                        # Create and save the base CSV file
clustering_results_base.to_csv('Clustering_Base_Features.csv', index=False)

                                                                                  # Define the clustering statistics columns (second file)
stats_columns = ['_id',
                 'KMeans1_Cluster', 'KMeans1_Silhouette_Score', 'KMeans1_Davies_Bouldin_Index', 'KMeans1_Calinski_Harabasz_Score', 'KMeans1_Prediction_Accuracy',
                 'DBSCAN1_Cluster', 'DBSCAN1_Silhouette_Score', 'DBSCAN1_Davies_Bouldin_Index', 'DBSCAN1_Prediction_Accuracy',
                 'KMeans2_Cluster', 'KMeans2_Silhouette_Score', 'KMeans2_Davies_Bouldin_Index', 'KMeans2_Calinski_Harabasz_Score', 'KMeans2_Prediction_Accuracy',
                 'DBSCAN2_Cluster', 'DBSCAN2_Silhouette_Score', 'DBSCAN2_Davies_Bouldin_Index', 'DBSCAN2_Prediction_Accuracy',
                 'KMeans3_Cluster', 'KMeans3_Silhouette_Score', 'KMeans3_Davies_Bouldin_Index', 'KMeans3_Calinski_Harabasz_Score', 'KMeans3_Prediction_Accuracy',
                 'DBSCAN3_Cluster', 'DBSCAN3_Silhouette_Score', 'DBSCAN3_Davies_Bouldin_Index', 'DBSCAN3_Prediction_Accuracy',
                 'KMeans4_Cluster', 'KMeans4_Silhouette_Score', 'KMeans4_Davies_Bouldin_Index', 'KMeans4_Calinski_Harabasz_Score', 'KMeans4_Prediction_Accuracy',
                 'DBSCAN4_Cluster', 'DBSCAN4_Silhouette_Score', 'DBSCAN4_Davies_Bouldin_Index', 'DBSCAN4_Prediction_Accuracy',
                 'KMeans5_Cluster', 'KMeans5_Silhouette_Score', 'KMeans5_Davies_Bouldin_Index', 'KMeans5_Calinski_Harabasz_Score', 'KMeans5_Prediction_Accuracy',
                 'DBSCAN5_Cluster', 'DBSCAN5_Silhouette_Score', 'DBSCAN5_Davies_Bouldin_Index', 'DBSCAN5_Prediction_Accuracy']
clustering_results_stats = clustering_results[stats_columns]                      # Create and save the clustering stats CSV file
clustering_results_stats.to_csv('Clustering_Result_Stats.csv', index=False)
files.download('Clustering_Result_Stats.csv')                                     # Download the clustering stats CSV file
display(HTML("""
    <p style="color: darkblue; font-size: 18px; font-weight: bold;">
         Clustering Base Features saved as <span style="color: darkblue;">'Clustering_Base_Features.csv'</span>.<br>
         Clustering Statistics saved as <span style="color: darkblue;">'Clustering_Result_Stats.csv'</span>.
    </p>
"""))
files.download('Best_Clustering_Models.csv')                                      # Download the clustering base CSV file
files.download('Clustering_Base_Features.csv')                                    # Download the clustering results CSV file
files.download('Clustering_Result_Stats.csv')                                     # Download the feature combo results CSV file

_id,EVENT_UNIQUE_ID,OCC_YEAR,OCC_MONTH,OCC_DAY,OCC_DOY,OCC_DOW,OCC_HOUR,DIVISION,LOCATION_TYPE,PREMISES_TYPE,HOOD_158,NEIGHBOURHOOD_158,LONG_WGS84,LAT_WGS84,OCC_DATETIME,reporting_delay_days,reporting_delay_hours,Location_Engineered,Hood_158_Encoded,Division_Encoded,Location_Engineered_Other,Location_Engineered_Public,Location_Engineered_Residential,OCC_Month_Encoded,OCC_DOW_Encoded,KMeans1_Cluster,KMeans1_Silhouette_Score,KMeans1_Davies_Bouldin_Index,KMeans1_Calinski_Harabasz_Score,KMeans1_Prediction_Accuracy,DBSCAN1_Cluster,DBSCAN1_Silhouette_Score,DBSCAN1_Davies_Bouldin_Index,DBSCAN1_Prediction_Accuracy,KMeans2_Cluster,KMeans2_Silhouette_Score,KMeans2_Davies_Bouldin_Index,KMeans2_Calinski_Harabasz_Score,KMeans2_Prediction_Accuracy,DBSCAN2_Cluster,DBSCAN2_Silhouette_Score,DBSCAN2_Davies_Bouldin_Index,DBSCAN2_Prediction_Accuracy,KMeans3_Cluster,KMeans3_Silhouette_Score,KMeans3_Davies_Bouldin_Index,KMeans3_Calinski_Harabasz_Score,KMeans3_Prediction_Accuracy,DBSCAN3_Cluster,DBSCAN3_Silhouette_Score,DBSCAN3_Davies_Bouldin_Index,DBSCAN3_Prediction_Accuracy,KMeans4_Cluster,KMeans4_Silhouette_Score,KMeans4_Davies_Bouldin_Index,KMeans4_Calinski_Harabasz_Score,KMeans4_Prediction_Accuracy,DBSCAN4_Cluster,DBSCAN4_Silhouette_Score,DBSCAN4_Davies_Bouldin_Index,DBSCAN4_Prediction_Accuracy,KMeans5_Cluster,KMeans5_Silhouette_Score,KMeans5_Davies_Bouldin_Index,KMeans5_Calinski_Harabasz_Score,KMeans5_Prediction_Accuracy,DBSCAN5_Cluster,DBSCAN5_Silhouette_Score,DBSCAN5_Davies_Bouldin_Index,DBSCAN5_Prediction_Accuracy
0,GO-20141263217,2013.0,December,31.0,365.0,Tuesday,17,D33,"Apartment (Rooming House, Condo)",Apartment,43,Victoria Village (43),-79.306754,43.734654,2013-12-31 17:00:00,0,23,Residential,0.004309,0.057760,0.0,0.0,1.0,12,5,1,0.653699,0.539332,58296.134469,65.369892,0,0.660512,0.518399,66.051157,1,0.639851,0.462713,90424.03486,63.985107,0,0.664183,0.554659,66.418339,1,0.644704,0.444792,103447.487598,64.470363,0,0.650294,0.531712,65.029381,1,0.862451,0.386864,220520.481791,86.2451,0,0.891933,0.114721,89.193326,2,0.700107,0.407972,126299.47312,70.010747,0,0.77939,0.340001,77.938967
1,GO-20141262914,2014.0,January,1.0,1.0,Wednesday,15,D43,"Streets, Roads, Highways (Bicycle Path, Private Road)",Outside,123,Cliffcrest (123),-79.236119,43.721827,2014-01-01 15:00:00,0,0,Public,0.004898,0.059435,0.0,1.0,0.0,1,6,0,0.653699,0.539332,58296.134469,65.369892,1,0.660512,0.518399,66.051157,2,0.639851,0.462713,90424.03486,63.985107,1,0.664183,0.554659,66.418339,3,0.644704,0.444792,103447.487598,64.470363,1,0.650294,0.531712,65.029381,0,0.862451,0.386864,220520.481791,86.2451,1,0.891933,0.114721,89.193326,0,0.700107,0.407972,126299.47312,70.010747,1,0.77939,0.340001,77.938967
2,GO-20141266097,2014.0,January,2.0,2.0,Thursday,1,D42,"Single Home, House (Attach Garage, Cottage, Mobile)",House,129,Agincourt North (129),-79.273925,43.813558,2014-01-02 01:00:00,0,7,Residential,0.009630,0.084440,0.0,0.0,1.0,1,4,1,0.653699,0.539332,58296.134469,65.369892,0,0.660512,0.518399,66.051157,1,0.639851,0.462713,90424.03486,63.985107,0,0.664183,0.554659,66.418339,1,0.644704,0.444792,103447.487598,64.470363,0,0.650294,0.531712,65.029381,1,0.862451,0.386864,220520.481791,86.2451,0,0.891933,0.114721,89.193326,2,0.700107,0.407972,126299.47312,70.010747,2,0.77939,0.340001,77.938967
3,GO-20141265947,2014.0,January,1.0,1.0,Wednesday,15,D23,"Parking Lots (Apt., Commercial Or Non-Commercial)",Outside,2,Mount Olive-Silverstone-Jamestown (2),-79.595344,43.744299,2014-01-01 15:00:00,0,17,Public,0.013754,0.129274,0.0,1.0,0.0,1,6,0,0.653699,0.539332,58296.134469,65.369892,1,0.660512,0.518399,66.051157,2,0.639851,0.462713,90424.03486,63.985107,1,0.664183,0.554659,66.418339,3,0.644704,0.444792,103447.487598,64.470363,1,0.650294,0.531712,65.029381,0,0.862451,0.386864,220520.481791,86.2451,1,0.891933,0.114721,89.193326,3,0.700107,0.407972,126299.47312,70.010747,3,0.77939,0.340001,77.938967
4,GO-20141265795,2014.0,January,1.0,1.0,Wednesday,19,D23,"Parking Lots (Apt., Commercial Or Non-

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

FileNotFoundError: Cannot find file: Best_Clustering_Models.csv